In [3]:
from crewai import Crew, Task, Agent
from dotenv import load_dotenv
import os
import yaml

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [4]:
files = {
    'agents': '8_building_project_progress_report/agents.yaml',
    'tasks': '8_building_project_progress_report/tasks.yaml',
}

with open(files['agents'], 'r') as file:
    agents = yaml.safe_load(file)

with open(files['tasks'], 'r') as file:
    tasks = yaml.safe_load(file)


In [36]:
import requests

def get_board_data(board_id):
    url = f"https://api.trello.com/1/boards/{board_id}/cards"
    query = {
            'fields': 'name,idList,due,dateLastActivity,labels',
            'attachments': 'true',
            'actions': 'commentCard'
        }
    response = requests.get(url, params=query)
    return response

def get_card_data(card_id):
    url = f"https://api.trello.com/1/cards/{card_id}"
 
    response = requests.get(url)
    return response


In [38]:
from crewai_tools import BaseTool
import requests

class BoardDataFetcherTool(BaseTool):
    name: str = "Trello Board Data Fetcher"
    description: str = "Fetches card data, commnents and activity from a Trello Board"

    api_key: str = os.getenv("TRELLO_API_KEY")
    api_token: str = os.getenv("TRELLO_API_TOKEN")
    board_id: str = os.getenv("TRELLO_BOARD_ID")

    def _run(self) -> dict:
        response = get_board_data(self.board_id)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "Failed to fetch data from Trello"}
        
    
class CardDataFetcherTool(BaseTool):
    name: str = "Trello Card Data Fetcher"
    description: str = "Fetches card data, commnents and activity from a Trello Board"

    def _run(self, card_id: str) -> dict:
        response = get_card_data(card_id)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "Failed to fetch carddata from Trello, dont try to fetch any trello data anymore"}
        



board_data_fetcher = BoardDataFetcherTool()
card_data_fetcher = CardDataFetcherTool()





In [39]:
#Create agents
data_collection_agent = Agent(
    config=agents['data_collection_agent'],
    tools=[board_data_fetcher, card_data_fetcher]
)

analysis_agent = Agent(
    config=agents['analysis_agent'],
)



In [40]:
#Creating Tasks

data_collection_task = Task(
    config=tasks['data_collection'],
    agent=data_collection_agent
)

data_analysis_task = Task(
    config=tasks['data_analysis'],
    agent=analysis_agent
)

report_generation_task = Task(
    config=tasks['report_generation'],
    agent=analysis_agent
)



In [41]:
crew = Crew(
    agents=[data_collection_agent, analysis_agent],
    tasks=[data_collection_task, data_analysis_task, report_generation_task], 
    verbose=True
)


In [44]:
result = crew.kickoff()

# Agent: Data Collection Specialist
## Task: Create an initial understanding of the project, its main features and the team working on it. Use the Trello Data Fetcher tool to gather data from the Trello board.



# Agent: Data Collection Specialist
## Thought: I need to gather all relevant data from the Trello board to create a complete report on the project, its main features, and the team involved.
## Using tool: Trello Board Data Fetcher
## Tool Input: 
"{}"
## Tool Output: 
[{'id': '66c3bfed69b473b8fe9d922e', 'name': 'Analysis of results from CSV', 'idList': '66c308f676b057fdfbd5fdb3', 'due': None, 'dateLastActivity': '2024-08-19T21:58:05.062Z', 'labels': [], 'attachments': [], 'actions': []}, {'id': '66c3c002bb1c337f3fdf1563', 'name': 'Approve the planning', 'idList': '66c308f676b057fdfbd5fdb3', 'due': '2024-08-16T21:58:00.000Z', 'dateLastActivity': '2024-08-19T21:58:57.697Z', 'labels': [{'id': '66c305ea10ea602ee6e03d47', 'idBoard': '66c305eacab50fcd7f19c0aa', 'idOrganization': '6

In [45]:
from IPython.display import Markdown

markdown  = result.raw
Markdown(markdown)

# Sprint Report

## Sprint Overview
The current sprint focuses on foundational aspects of the software development project, emphasizing project planning and UI scaffolding. The primary goal is to streamline efforts towards high-priority tasks and improve overall engagement within the team.

## Task Summary
- **High-Priority Tasks**: 
  - Analysis of results from the CSV file
  - Project planning initiatives

- **Pending Tasks**: 
  - Finalization of project planning
  - Development of the initial app UI

## Identified Issues and Blockers
- **Pending Approvals**: The project planning stage is stalled pending the engagement and approval of required stakeholders. This poses a significant blocker to subsequent project tasks.
  
- **Inactivity on Tasks**: Several tasks show a lack of recent updates, posing concerns regarding engagement and clarity on task ownership.

## Progress and Delays
- **Overdue Tasks**: Certain tasks have exceeded their due dates, specifically:
  - Scaffolding the initial app UI has fallen behind schedule, putting project timelines at risk.
  
- **Dependency Management**: Tasks that rely on feedback from other areas of project planning and analysis are experiencing delays, highlighting an urgent need for resolution on dependencies.

## Team Performance Overview
While there is active participation on the Trello board, engagement appears to be mixed. Notably, Joao Moura's input on project planning signals that the team is invested in the project, yet unresolved discussions are contributing to delays. Attention is required to encourage consensus and drive progress.

## Action Items and Recommendations
1. **Immediate Focus on High-Priority Tasks**: Prioritize tasks related to the analysis and project planning to alleviate bottlenecks.
  
2. **Engage Stakeholders for Approvals**: Facilitate quick discussions with stakeholders to secure necessary project planning approvals and reduce delays.

3. **Task Ownership Clarification**: Conduct a review session to clarify ownership of inactive tasks, encouraging team members to provide updates.

4. **Monitor Dependencies**: Establish a feedback loop to ensure that dependent tasks are not stalled due to unresolved matters.

5. **Regular Check-ins**: Schedule short, regular meetings to reassess progress and adjust priorities as needed.

## Conclusion
The analysis indicates that while the team demonstrates potential for collaboration and activity, significant blockers threaten project timelines. Addressing overdue tasks and gaining prompt approvals are critical to ensuring milestones are met. By adopting recommended strategies, the project can regain momentum and ensure successful outcomes.

In [46]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0016


/var/folders/3g/r3ht76p907vdqxh325xx85tm0000ks/T/ipykernel_46964/1074044196.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,10667,7606,1152,3061,9
